In [26]:
import json
from urllib.request import urlopen
import requests,json,os,sys,time,re
import pandas as pd
import numpy as np

In [27]:
def show_work_status(singleCount, totalCount, currentCount):
    currentCount+=singleCount
    percentage=1.*currentCount/totalCount*100
    status='>'*int(percentage/2)+'-'*(50-int(percentage/2))
    if percentage % 3:
        sys.stdout.write('\rStatus:[{0}]{1:.2f}%'.format(status,percentage))
        sys.stdout.flush()
    if percentage>=100: print('\n')

In [28]:
content_data = pd.read_csv('new_steam_games.csv')
content_data

,Positive,Negative,Game_name,Positive_Rate,Link
0,39146,3404,DOOM,0.92,https://store.steampowered.com/app/379720/DOOM/
1,409937,426671,PLAYERUNKNOWN'S BATTLEGROUNDS,0.49,https://store.steampowered.com/app/578080/PLAY...
2,4991,2039,BATTLETECH,0.71,https://store.steampowered.com/app/637090/BATT...
3,101940,65175,DayZ,0.61,https://store.steampowered.com/app/221100/DayZ/
4,8495,2986,EVE Online,0.74,https://store.steampowered.com/app/8500/EVE_On...
...,...,...,...,...,...
17358,17,0,Grabity,1.00,https://store.steampowered.com/app/652810/Grab...
17359,10,4,Detached: Non-VR Edition,0.78,https://store.steampowered.com/app/848410/Deta...
17360,9,2,A Room Beyond,0.90,https://store.steampowered.com/app/454330/A_Ro...
17361,7,3,Chasing the Stars,0.70,https://store.steampowered.com/app/763990/Chas...


In [29]:
import urllib
import bs4
from bs4 import BeautifulSoup
#url_contents = urllib.request.urlopen(url).read()
Game_description = pd.Series([]) 
current_count = 0
for i in range(100):
    url = content_data['Link'][i]
    url_contents = urllib.request.urlopen(url).read()
    soup = bs4.BeautifulSoup(url_contents, 'html.parser')
    div = soup.find("div", {"id": "game_area_description"})
    ########## Fina the area of game descriptiona ##########
    Game_description[i]=div
    show_work_status(1,100,current_count)
    current_count+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [30]:
soup = bs4.BeautifulSoup(url_contents, 'html.parser')
div = soup.find("div", {"id": "game_area_description"})

content = str(div)
content_data['Game_description'] = Game_description
content_data['Game_description'] = content_data['Game_description'].apply(str)
content_data

,Positive,Negative,Game_name,Positive_Rate,Link,Game_description
0,39146,3404,DOOM,0.92,https://store.steampowered.com/app/379720/DOOM/,"<div class=""game_area_description"" id=""game_ar..."
1,409937,426671,PLAYERUNKNOWN'S BATTLEGROUNDS,0.49,https://store.steampowered.com/app/578080/PLAY...,"<div class=""game_area_description"" id=""game_ar..."
2,4991,2039,BATTLETECH,0.71,https://store.steampowered.com/app/637090/BATT...,"<div class=""game_area_description"" id=""game_ar..."
3,101940,65175,DayZ,0.61,https://store.steampowered.com/app/221100/DayZ/,"<div class=""game_area_description"" id=""game_ar..."
4,8495,2986,EVE Online,0.74,https://store.steampowered.com/app/8500/EVE_On...,"<div class=""game_area_description"" id=""game_ar..."
...,...,...,...,...,...,...
17358,17,0,Grabity,1.00,https://store.steampowered.com/app/652810/Grab...,nan
17359,10,4,Detached: Non-VR Edition,0.78,https://store.steampowered.com/app/848410/Deta...,nan
17360,9,2,A Room Beyond,0.90,https://store.steampowered.com/app/454330/A_Ro...,nan
17361,7,3,Chasing the Stars,0.70,https://store.steampowered.com/app/763990/Chas...,nan


In [31]:
print(content_data['Game_description'][1])

<div class="game_area_description" id="game_area_description">
<h2>About This Game</h2>
<strong>LAND, LOOT, SURVIVE!</strong><br/>Play PUBG: BATTLEGROUNDS for free.<br/>Land on strategic locations, loot weapons and supplies, and survive to become the last team standing across various, diverse Battlegrounds.<br/>Squad up and join the Battlegrounds for the original Battle Royale experience that only PUBG: BATTLEGROUNDS can offer.<br/><br/>This content download will also provide access to the BATTLEGROUNDS Test Server, which requires a separate download to play. <br/>Optional in-game purchases available.						</div>


In [32]:
import string
current_count = 0
for i in range(0,100):
    content_data['Game_description'][i] = re.sub('<.*?>', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = re.sub('\\n.*\\n', ' ', content_data['Game_description'][i])
    content_data['Game_description'][i] = content_data['Game_description'][i].replace("\t","")
    content_data['Game_description'][i] = content_data['Game_description'][i].translate(content_data['Game_description'][i].maketrans(' ',' ',string.punctuation))

    show_work_status(1,100,current_count)
    current_count+=1

Status:[>>>>>>>>>>>>>>>>>>>-------------------------------]38.00%

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: Se

Status:[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%



In [33]:
content_data.head(5)

,Positive,Negative,Game_name,Positive_Rate,Link,Game_description
0,39146,3404,DOOM,0.92,https://store.steampowered.com/app/379720/DOOM/,Developed by id software the studio that pio...
1,409937,426671,PLAYERUNKNOWN'S BATTLEGROUNDS,0.49,https://store.steampowered.com/app/578080/PLAY...,LAND LOOT SURVIVE Play PUBG BATTLEGROUNDS ...
2,4991,2039,BATTLETECH,0.71,https://store.steampowered.com/app/637090/BATT...,From original BATTLETECHMechWarrior creat...
3,101940,65175,DayZ,0.61,https://store.steampowered.com/app/221100/DayZ/,DayZ is a hardcore openworld survival game w...
4,8495,2986,EVE Online,0.74,https://store.steampowered.com/app/8500/EVE_On...,THE 1 SPACE MMO EVE Online is the largest...


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(list(content_data['Game_description']))
#Need to convert every Series in DataFrame to List format
tfidf_vectorizer.get_feature_names()
#Array mapping from feature integer indices to feature name
#print(tfidf)  ####Tf-idf-weighted document-term matrix.     (0, 3473)	0.4059439800797409   (0, 1417)	0.08671197437269683

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['0100',
 '10',
 '100',
 '1000',
 '10000',
 '1000yearold',
 '1080p',
 '10th',
 '11',
 '110',
 '1100',
 '1142',
 '12',
 '13',
 '130',
 '13th',
 '14',
 '14th',
 '15',
 '16',
 '160',
 '17',
 '170',
 '1796',
 '18',
 '180',
 '1802',
 '1803',
 '1807',
 '18ft',
 '19',
 '1912',
 '1984',
 '1988',
 '1999',
 '1realistic',
 '1v1',
 '20',
 '200',
 '20000',
 '2015',
 '2020',
 '2021',
 '2022',
 '20mm',
 '20th',
 '21',
 '22',
 '23',
 '230',
 '2300',
 '2478',
 '25',
 '250',
 '26',
 '27',
 '28',
 '2d',
 '2e',
 '2nd',
 '2various',
 '2x',
 '30',
 '300',
 '3025',
 '34',
 '35',
 '350',
 '35e',
 '36',
 '38',
 '3d',
 '3dmark',
 '3rd',
 '3unique',
 '3vs3',
 '40',
 '400',
 '40000',
 '45',
 '48',
 '4e',
 '4k',
 '4multiple',
 '50',
 '500',
 '5000',
 '501st',
 '51',
 '510',
 '5e',
 '60',
 '600',
 '6000',
 '60fps',
 '60man',
 '64',
 '64bit',
 '65',
 '650000',
 '67',
 '6player',
 '6v6',
 '70',
 '700',
 '7000',
 '7x',
 '80',
 '800',
 '80s',
 '840',
 '86',
 '865',
 '867',
 '870',
 '872',
 '873',
 '877',
 '878',
 '8825

In [35]:
list_app_name = list(content_data['Game_name'])
content_data['Game_description'] 

0          Developed by id software the studio that pio...
1           LAND LOOT SURVIVE  Play PUBG BATTLEGROUNDS ...
2             From original BATTLETECHMechWarrior creat...
3          DayZ is a hardcore openworld survival game w...
4           THE 1 SPACE MMO   EVE Online is the largest...
                               ...                        
17358                                                  nan
17359                                                  nan
17360                                                  nan
17361                                                  nan
17362                                                  nan
Name: Game_description, Length: 17363, dtype: object

In [36]:
dic_recommended_1 = {}
for index in range(content_data['Game_description'].shape[0]):
    #Iterate items in DataFrame
    similarities_1 = linear_kernel(tfidf[index],tfidf).flatten()
    ####For each game，tfidf value list is a very long list，tfidf[index:index+1] calculates all of the words'tfidf for one game,
    related_docs_indices_1 = (-similarities_1).argsort()[1:11]
    #related_docs_indices_2 = similarities_2.argsort()[:-10:-1]
    #argsort returns an array of indices of the same shape as 
    dic_recommended_1.update({list_app_name[index]:[list_app_name[i] for i in related_docs_indices_1]})
df_content_based_results_1 = pd.DataFrame(dic_recommended_1)
#df_content_based_results_1.index.name = 'steam_appname'
df_content_based_results_1.reset_index(inplace=True)

In [37]:
df_content_based_results_1 = df_content_based_results_1.T
print(df_content_based_results_1.shape)
df_content_based_results_1

(17350, 10)


,0,1,2,3,4,5,6,7,8,9
index,0,1,2,3,4,5,6,7,8,9
DOOM,Golf It!,Call of Duty®: WWII,Dungeons & Dragons Online®,Clone Drone in the Danger Zone,Call of Duty®: Black Ops,Men of War: Assault Squad 2,The Witcher: Enhanced Edition Director's Cut,Forts,Devil May Cry 5,ShellShock Live
PLAYERUNKNOWN'S BATTLEGROUNDS,Realm Royale,Northgard,Desert Skies,Dungeons & Dragons Online®,The Elder Scrolls®: Legends™,EVE Online,Keep Talking and Nobody Explodes,Call of Duty®: WWII,The Surge,For The King
BATTLETECH,Battle Brothers,Heroes & Generals,Total War: MEDIEVAL II – Definitive Edition,Call of Duty®: WWII,War Robots,Men of War: Assault Squad 2,STAR WARS™ Empire at War - Gold Pack,Between the Stars,Call of Duty®: Black Ops,For The King
DayZ,They Are Billions,Frostpunk,The Evil Within,The Jackbox Party Pack 2,Stonehearth,Neverwinter Nights: Enhanced Edition,GOD WARS The Complete Legend,Dungeons & Dragons Online®,Clone Drone in the Danger Zone,Call of Duty®: Black Ops
...,...,...,...,...,...,...,...,...,...,...
Grabity,Battle Fleet: Ground Assault,Bass Blocks,The Deer,Kanji Training Game,Star Conflict: Fleet Strength - Mauler,Just Cause™ 3 DLC: Mech Land Assault,Star Conflict: Fleet Strength - Razor,What Are You Stupid,Distant Worlds: Universe,The Dark Inside Me
Detached: Non-VR Edition,Battle Fleet: Ground Assault,Bass Blocks,The Deer,Kanji Training Game,Star Conflict: Fleet Strength - Mauler,Just Cause™ 3 DLC: Mech Land Assault,Star Conflict: Fleet Strength - Razor,What Are You Stupid,Distant Worlds: Universe,The Dark Inside Me
A Room Beyond,Battle Fleet: Ground Assault,Bass Blocks,The Deer,Kanji Training Game,Star Conflict: Fleet Strength - Mauler,Just Cause™ 3 DLC: Mech Land Assault,Star Conflict: Fleet Strength - Razor,What Are You Stupid,Distant Worlds: Universe,The Dark Inside Me
Chasing the Stars,Battle Fleet: Ground Assault,Bass Blocks,The Deer,Kanji Training Game,Star Conflict: Fleet Strength - Mauler,Just Cause™ 3 DLC: Mech Land Assault,Star Conflict: Fleet Strength - Razor,What Are You Stupid,Distant Worlds: Universe,The Dark Inside Me


In [38]:
print(content_data['Game_name'].tolist())

['DOOM', "PLAYERUNKNOWN'S BATTLEGROUNDS", 'BATTLETECH', 'DayZ', 'EVE Online', 'Devil May Cry 5', 'Human: Fall Flat', 'They Are Billions', 'Warhammer: Chaosbane', 'For The King', 'Danganronpa V3: Killing Harmony', 'TERA', 'Call of Duty®: Modern Warfare® Remastered', 'Stonehearth', 'Clone Drone in the Danger Zone', 'GOD EATER 3', 'War Robots', 'Halo Wars: Definitive Edition', 'Call of Duty®: Black Ops', 'Phoenix Wright: Ace Attorney Trilogy / 逆転裁判123 成歩堂セレクション', 'Team Sonic Racing™', 'Titan Quest Anniversary Edition', 'Grand Theft Auto IV', 'Portal', 'Viscera Cleanup Detail', 'Neverwinter Nights: Enhanced Edition', 'Ni no Kuni™ II: Revenant Kingdom', 'Tree of Savior (English Ver.)', "Hellblade: Senua's Sacrifice", 'Danganronpa 2: Goodbye Despair', 'Yakuza 0', 'The Escapists 2', 'Star Wars: Battlefront 2 (Classic, 2005)', 'Wizard101', 'Life is Strange 2', 'Kingdoms of Amalur: Reckoning™', 'GOD WARS The Complete Legend', 'Counter-Strike: Source', 'The Evil Within', 'Bright Memory(Early Acc

Example Output

In [39]:
user_profile = [["DARK SOULS III", 286], ['Grand Theft Auto V', 45], ['Portal 2', 8], ['RESIDENT EVIL 7 biohazard / BIOHAZARD 7 resident evil', 88], ['Warface', 1.4]]


In [40]:
names = []
for i in range(len(user_profile)):
  names.append(user_profile[i][0])
  if user_profile[i][1] <= 2:
    user_profile[i][1] = 0.1
  elif user_profile[i][1] > 2 and user_profile[i][1] <= 10:
    user_profile[i][1] = 0.5
  elif user_profile[i][1] > 10 and user_profile[i][1] <= 50:
    user_profile[i][1] = 0.8
  elif user_profile[i][1] > 50 and user_profile[i][1] <= 100: 
    user_profile[i][1] = 1
  else:
    user_profile[i][1] = 1.2
user_profile

[['DARK SOULS III', 1.2],
 ['Grand Theft Auto V', 0.8],
 ['Portal 2', 0.5],
 ['RESIDENT EVIL 7 biohazard / BIOHAZARD 7 resident evil', 1],
 ['Warface', 0.1]]

In [41]:
recommendations = {}
for game in user_profile:
  name = game[0]
  id = content_data.index[content_data['Game_name'] == name][0]
  similarities = linear_kernel(tfidf[id],tfidf).flatten()
  related_docs_indices = (-similarities).argsort()[1:11]
  lam = 0
  for i in related_docs_indices:
    new_game = list_app_name[i]
    if new_game not in names:
      if new_game not in recommendations:
        recommendations.update({new_game: game[1]-lam})
      else:
        value = recommendations.get(new_game)
        recommendations.update({new_game: value+game[1]-lam})
    lam += 0.01

recommendations = dict(sorted(recommendations.items(), key=lambda item: item[1], reverse=True))
print(recommendations)

{'Battle Fleet: Ground Assault': 3.6, 'Bass Blocks': 3.5500000000000007, 'The Deer': 3.5, 'Kanji Training Game': 3.4500000000000006, 'Star Conflict: Fleet Strength - Mauler': 3.4, 'Just Cause™ 3 DLC: Mech Land Assault': 3.3500000000000005, 'Star Conflict: Fleet Strength - Razor': 3.3, 'What Are You Stupid': 3.2500000000000004, 'Distant Worlds: Universe': 3.1999999999999997, 'The Dark Inside Me': 3.1500000000000004}


In [42]:
output = []
count = 0
for key in recommendations:
  output.append(key)
  count += 1
  if count >= 10:
    break
output

['Battle Fleet: Ground Assault',
 'Bass Blocks',
 'The Deer',
 'Kanji Training Game',
 'Star Conflict: Fleet Strength - Mauler',
 'Just Cause™ 3 DLC: Mech Land Assault',
 'Star Conflict: Fleet Strength - Razor',
 'What Are You Stupid',
 'Distant Worlds: Universe',
 'The Dark Inside Me']